# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.78,88,0,7.77,PN,1728096718
1,1,cambria,35.5641,-121.0808,15.59,95,58,0.45,US,1728096720
2,2,roma,43.2128,-75.4557,16.08,91,100,2.06,US,1728096686
3,3,yellowknife,62.4560,-114.3525,-0.71,93,20,1.54,CA,1728096723
4,4,dunedin,-45.8742,170.5036,13.22,99,99,0.89,NZ,1728096583


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Assuming you have already imported hvplot.pandas and other necessary libraries

# Step 1: Create the map plot
# Configure the map plot using hvplot
map_plot = city_data_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    size='Humidity',            # Use the Humidity column to set the size of the points
    color='blue',              # Choose a color for the points
    title='City Humidity Levels',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=800,          # Width of the frame
    frame_height=600          # Height of the frame
)

# Step 2: Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define the ideal weather conditions
min_temp = 20  # Minimum temperature in Celsius
max_temp = 30  # Maximum temperature in Celsius
max_humidity = 50  # Maximum humidity in percentage
max_cloudiness = 0  # Maximum cloud cover in percentage
max_wind_speed = 10  # Maximum wind speed in meters per second

# Narrow down cities that fit the criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) &
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Cloudiness"] == max_cloudiness) &
    (city_data_df["Wind Speed"] <= max_wind_speed)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data (formatted as per your request)
ideal_cities_df = ideal_cities_df.reset_index(drop=True)
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,27,mhamid,29.8200,-5.7200,27.98,32,0,3.80,MA,1728096759
1,70,ha'il,27.5219,41.6907,23.01,35,0,2.06,SA,1728096825
2,112,rawlins,41.7911,-107.2387,20.20,10,0,2.57,US,1728096916
3,151,newman,37.3138,-121.0208,28.36,39,0,0.89,US,1728096977
4,165,nuevo casas grandes,30.4167,-107.9167,20.37,22,0,2.91,MX,1728096997


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the copy function to create a new DataFrame for storing the required data
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mhamid,MA,29.8200,-5.7200,32,
1,ha'il,SA,27.5219,41.6907,35,
2,rawlins,US,41.7911,-107.2387,10,
3,newman,US,37.3138,-121.0208,39,
4,nuevo casas grandes,MX,30.4167,-107.9167,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters radius
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()




 

Starting hotel search
mhamid - nearest hotel: Hotel Kasbah Azalay
ha'il - nearest hotel: فندق كلاسيك
rawlins - nearest hotel: Quality Inn Rawlins I-80
newman - nearest hotel: No hotel found
nuevo casas grandes - nearest hotel: Hotel Paquime
kandahar - nearest hotel: No hotel found
isna - nearest hotel: No hotel found
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
jirja - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
tozkhurmato - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
tamanrasset - nearest hotel: فندق الأمان


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mhamid,MA,29.8200,-5.7200,32,Hotel Kasbah Azalay
1,ha'il,SA,27.5219,41.6907,35,فندق كلاسيك
2,rawlins,US,41.7911,-107.2387,10,Quality Inn Rawlins I-80
3,newman,US,37.3138,-121.0208,39,No hotel found
4,nuevo casas grandes,MX,30.4167,-107.9167,22,Hotel Paquime


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Import hvplot and geoviews
import hvplot.pandas
import geoviews as gv
from geoviews import opts

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',  # Use Humidity to size the points
    color='Humidity',  # Color based on humidity levels
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add city, country, and hotel name to the hover tooltip
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=800,
    frame_height=600,
    title='Hotel Locations based on Ideal Weather Conditions'
)

# Display the map
hotel_map.opts(
    opts.Points(
        tools=['hover'],  # Enable hover tool
        colorbar=True,
        size=10,
        cmap='Viridis',
    )
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)